In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
import os
import kagglehub
import shutil
import requests
import gzip
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
DATASETS = [
    "shivamb/amazon-prime-movies-and-tv-shows",
    "shivamb/disney-movies-and-tv-shows",
    "shivamb/netflix-shows",
]

current_directory = os.getcwd() 
DESTINATION_DIR = os.path.join(os.path.join(current_directory, '..'), "data") 

# 1. Cria a pasta 'data' se ela não existir
os.makedirs(DESTINATION_DIR, exist_ok=True)

def download_and_copy_dataset(dataset_name: str, destination_path: str):
    """Baixa um dataset, copia os arquivos para o destino e limpa o cache."""

    # 1. Baixa o dataset para o cache
    try:
        cache_path = kagglehub.dataset_download(dataset_name)
    except Exception as e:
        return

    # 2. Copia os arquivos do cache para o diretório de destino
    for item_name in os.listdir(cache_path):
        source = os.path.join(cache_path, item_name)
        destination = os.path.join(destination_path, item_name)

        # Copia apenas arquivos (ignorando subpastas)
        if os.path.isfile(source):
            shutil.copy2(source, destination) 

    # 3. Remove completamente a pasta do cache
    try:
        shutil.rmtree(cache_path)
    except OSError as e:
        print(f"  > AVISO: Não foi possível remover o cache: {e}")
        
for dataset in DATASETS:
    download_and_copy_dataset(dataset, DESTINATION_DIR)

print("\n\n--- Processo Finalizado ---")
print(f"Todos os arquivos dos datasets estão na pasta: {DESTINATION_DIR}")

In [ ]:
def baixar_e_descomprimir_imdb(output_dir="../data"):
    """
    Baixa os arquivos de dataset do IMDb e os descomprime.
    Os arquivos são salvos no diretório 'output_dir'.
    """
    
    # URL base dos datasets
    base_url = "https://datasets.imdbws.com/"
    
    # Arquivos necessários para (Nome da Obra, Nota, Diretor)
    files_to_download = [
        "title.basics.tsv.gz",   # Mapeia tconst -> primaryTitle (Nome da Obra)
        "title.ratings.tsv.gz",  # Mapeia tconst -> averageRating (Nota)
        "title.crew.tsv.gz",     # Mapeia tconst -> nconst (ID do Diretor)
        "name.basics.tsv.gz"     # Mapeia nconst -> primaryName (Nome da Pessoa)
    ]
    
    # Cria o diretório de saída se ele não existir
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Diretório criado: '{output_dir}'")

    print("Iniciando downloads... (Isso pode levar vários minutos por arquivo)")

    for filename in files_to_download:
        url = base_url + filename
        
        gz_path = os.path.join(output_dir, filename)
        
        tsv_path = gz_path.replace(".gz", "")
        
        try:
            print(f"\nBaixando: {filename}...")
            with requests.get(url, stream=True) as r:
                r.raise_for_status() # Lança um erro se o status não for 200
                with open(gz_path, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192): 
                        f.write(chunk)
            print("Download concluído.")

            print(f"Descomprimindo: {filename}...")
            with gzip.open(gz_path, 'rb') as f_in:
                with open(tsv_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
            print(f"Arquivo salvo: {tsv_path}")
            
            os.remove(gz_path)
            print(f"Arquivo temporário '{gz_path}' removido.")

        except requests.exceptions.RequestException as e:
            print(f"\nERRO: Falha ao baixar {url}. Motivo: {e}")
            print("Por favor, verifique sua conexão ou a URL.")
            if os.path.exists(gz_path):
                os.remove(gz_path)
        except Exception as e:
            print(f"\nERRO: Ocorreu um problema: {e}")

    print(f"\nProcesso concluído! Os arquivos TSV estão em: '{output_dir}'")

baixar_e_descomprimir_imdb()

In [ ]:
#Concatenar as base de dados 

current_directory = os.getcwd() 
DESTINATION_DIR = os.path.join(os.path.join(current_directory, '..'), "data") 

df_netflix = pd.read_csv(os.path.join(DESTINATION_DIR, 'netflix_titles.csv'))
df_disney = pd.read_csv(os.path.join(DESTINATION_DIR, 'disney_plus_titles.csv'))
df_amazon = pd.read_csv(os.path.join(DESTINATION_DIR, 'amazon_prime_titles.csv'))

df_netflix['streaming'] = 'Netflix'
df_disney['streaming'] = 'Disney+'
df_amazon['streaming'] = 'Prime Video'

dataframes_to_concat = [df_netflix, df_disney, df_amazon]

df_streaming = pd.concat(dataframes_to_concat, ignore_index=True)

df_streaming['date_added'] = df_streaming['date_added'].str.strip()

df_streaming['date_added'] = pd.to_datetime(df_streaming['date_added'], format='%B %d, %Y')

# Exibindo informações gerais para confirmar a junção e os tipos de dados
print("\nInformações do DataFrame consolidado:")
df_streaming.head(5)

In [ ]:
genre_mapping = {
    # --- PADRONIZAÇÃO E CONSOLIDAÇÃO (TV/Filmes -> Gênero Principal) ---
    'Dramas': 'Drama', 'TV Dramas': 'Drama',
    'Comedies': 'Comedy', 'TV Comedies': 'Comedy', 'Romantic Comedy': 'Romance , Comedy',
    'Thrillers': 'Thriller', 'TV Thrillers': 'Thriller',
    'Documentaries': 'Documentary', 'Docuseries': 'Documentary',
    'Horror Movies': 'Horror', 'TV Horror': 'Horror',
    'Romantic Movies': 'Romance', 'Romantic TV Shows': 'Romance',
    'International Movies': 'International', 'International TV Shows': 'International',
    'Independent Movies': 'Independent',
    'Sports Movies': 'Sports',
    'Classic Movies': 'Classic',
    'Cult Movies': 'Cult',
    'LGBTQ Movies': 'LGBTQ',
    'Crime TV Shows': 'Crime',
    'Reality TV': 'Reality',
    'Teen TV Shows': 'Teen',
    'TV Mysteries': 'Mystery',
    'Science Fiction': 'Sci-Fi',

    # --- MAPEAMENTO DE SINÔNIMOS E SUB-GÊNEROS ---
    'Anime Features': 'Anime', 'Anime Series': 'Anime',
    "Kids' TV": "Kids", 'Children & Family Movies': 'Kids, Family',
    'Faith and Spirituality': 'Faith & Spirituality',
    'Young Adult Audience': 'Young Adult',
    'Soap Opera / Melodrama': 'Soap Opera',
    'and Culture': 'Culture', 

    # --- SEPARAÇÃO DE GÊNEROS COMPOSTOS (usando vírgula) ---
    'Animals & Nature': 'Nature',
    'Science & Nature': 'Nature, Science', 
    'Arts & Culture': 'Culture, Art',
    'Action & Adventure': 'Action, Adventure',
    'Sci-Fi & Fantasy': 'Sci-Fi, Fantasy',
    'Stand-Up Comedy & Talk Shows': 'Stand-Up Comedy, Talk Show',
    'Music Videos and Concerts': 'Music',
    'Music & Musicals': 'Music, Musical',
    'Science & Nature TV': 'Science, Nature',
    'Animals & Nature': 'Animals, Nature',
    'TV Action & Adventure': 'Action, Adventure',
    'TV Sci-Fi & Fantasy': 'Sci-Fi, Fantasy',
    'Game Show / Competition': 'Game Show, Competition',
    'Action-Adventure': 'Action, Adventure',
    'Classic & Cult TV': 'Classic, Cult',
    'Talk Show and Variety': 'Talk Show, Variety',
    
    # --- Mapeamento direto de gêneros de TV para manter a distinção se desejado ---
    'Korean TV Shows': 'Korean TV',
    'British TV Shows': 'British TV',
    'Spanish-Language TV Shows': 'Spanish TV',

    # --- Remoção de Formatos (não são gêneros temáticos) ---
    'Movies': '_REMOVE_',
    'Series': '_REMOVE_',
    'TV Shows': '_REMOVE_', 
    'TV Show': '_REMOVE_',
    'Anthology': '_REMOVE_',
    'Unscripted': '_REMOVE_', # Categoria muito ampla, coberta por Reality
    'Special Interest': '_REMOVE_' # Categoria muito genérica
}

def process_genres(genre_string):
    """
    Função para aplicar o mapeamento de gênero em uma string 
    que pode conter múltiplos gêneros.
    """
    if pd.isna(genre_string):
        return '' # Retorna string vazia 

    processed_genres = set()
    
    # 1. Separa os gêneros da string original (ex: "TV Dramas, TV Mysteries")
    initial_genres = genre_string.split(',')

    for genre in initial_genres:
        # 2. Limpa o whitespace (ex: " TV Dramas" -> "TV Dramas")
        clean_genre = genre.strip()

        # 3. Aplica o mapping. 
        mapped_value = genre_mapping.get(clean_genre, clean_genre)

        # 4. Processa o valor mapeado
        if mapped_value == '_REMOVE_':
            # Não faz nada, simplesmente ignora o gênero
            continue
        elif ',' in mapped_value:
            # Separa, limpa e adiciona cada sub-gênero
            sub_genres = mapped_value.split(',')
            for sub in sub_genres:
                processed_genres.add(sub.strip())
        else:
            # É um valor único, não vazio e não _REMOVE_
            if mapped_value:
                processed_genres.add(mapped_value)
    
    # ordenados alfabeticamente para consistência.
    return ', '.join(sorted(list(processed_genres)))

df_streaming['genres_processed'] = df_streaming['listed_in'].apply(process_genres)

# 4. (Opcional) Mostra o resultado das 10 primeiras linhas
print("Processamento concluído. Exemplo do resultado:")
print(df_streaming[['listed_in', 'genres_processed']].head(10))

print(df_streaming.dtypes)

In [ ]:
output_csv_file = os.path.join('../data', 'filmes_series.csv')
df_streaming.to_csv(output_csv_file, index=False)

In [ ]:
imdb_data_dir = '../data'
basics_tsv = os.path.join(imdb_data_dir, 'title.basics.tsv')
ratings_tsv = os.path.join(imdb_data_dir, 'title.ratings.tsv')
crew_tsv = os.path.join(imdb_data_dir, 'title.crew.tsv')
names_tsv = os.path.join(imdb_data_dir, 'name.basics.tsv')
movies_csv_file = os.path.join(imdb_data_dir, 'filmes_series.csv')
output_csv_file = os.path.join(imdb_data_dir, 'filmes_series_imdb.csv')

con = duckdb.connect(database=':memory:', read_only=False)

# --- Consulta SQL Principal ---
# Esta consulta é longa, pois prepara e junta 5 arquivos diferentes.
# Consulta desenvolvida com auxilio de IA
merge_query = f"""
WITH
-- 1. Prepara a base de Títulos e Notas do IMDb
imdb_titles_with_rating AS (
    SELECT
        basics.tconst,
        basics.primaryTitle,
        basics.titleType,
        basics.runtimeMinutes,
        ratings.averageRating,
        ratings.numVotes,
        TRY_CAST(basics.startYear AS INT64) AS startYear
    FROM read_csv_auto('{basics_tsv}', header=True, delim='\t', quote='', strict_mode=False) AS basics
    JOIN read_csv_auto('{ratings_tsv}', header=True, delim='\t', quote='') AS ratings
        ON basics.tconst = ratings.tconst
    WHERE basics.titleType IN ('movie', 'tvSeries', 'tvMiniSeries', 'tvMovie')
),

-- 2. Prepara a base de Diretores do IMDb 
imdb_director_names AS (
    SELECT
        crew.tconst,
        names.primaryName AS imdb_director_name
    FROM read_csv_auto('{crew_tsv}', header=True, delim='\t', quote='') AS crew
    CROSS JOIN unnest(string_split(crew.directors, ',')) AS t(nconst_id)
    JOIN read_csv_auto('{names_tsv}', header=True, delim='\t', quote='', strict_mode=False) AS names
        ON names.nconst = t.nconst_id
    WHERE t.nconst_id != '\\N'
),

-- 3. Junta Títulos, Notas e Nomes de Diretores do IMDb 
imdb_full_directors AS (
    SELECT
        LOWER(TRIM(t.primaryTitle)) AS imdb_title_clean,
        LOWER(TRIM(d.imdb_director_name)) AS imdb_director_clean,
        t.averageRating AS imdb_rating,
        t.runtimeMinutes AS run_time_minutes,
        t.numVotes AS number_votes
    FROM imdb_titles_with_rating AS t
    JOIN imdb_director_names AS d ON t.tconst = d.tconst
    GROUP BY 1, 2, 3, 4, 5
),

-- 4. Prepara a base de dados (sem alteração, com UNION ALL)
netflix_directors_exploded AS (
    SELECT
        n.*, 
        LOWER(TRIM(director_name)) AS director_clean,
        LOWER(TRIM(title)) AS title_clean
    FROM read_csv_auto('{movies_csv_file}', header=True, auto_detect=True) AS n
    CROSS JOIN unnest(string_split(n.director, ',')) AS t(director_name)
    WHERE n.director IS NOT NULL
    UNION ALL
    SELECT
        n.*, 
        NULL AS director_clean, 
        LOWER(TRIM(title)) AS title_clean
    FROM read_csv_auto('{movies_csv_file}', header=True, auto_detect=True) AS n
    WHERE n.director IS NULL
),

-- 5. Prepara a base do IMDb para junção por Título + Ano
imdb_full_year AS (
    SELECT
        LOWER(TRIM(primaryTitle)) AS imdb_title_clean,
        startYear,
        AVG(averageRating) AS imdb_rating -- Média caso haja duplicatas (raro)
    FROM imdb_titles_with_rating
    GROUP BY 1, 2
)


-- 6. Consulta Final: Junta dataset com IMDb 
SELECT
    n.show_id,
    n.type,
    n.title,
    n.director,
    n.cast,
    n.country,
    n.date_added,
    n.release_year,
    n.rating,
    n.duration,
    n.listed_in,
    n.description,
    n.genres_processed,
    n.streaming,
    
    -- COALESCE usa a primeira nota que não for NULA
    COALESCE(
        AVG(i_director.imdb_rating), -- 1ª Tentativa: Título + Diretor
        AVG(i_year.imdb_rating)      -- 2ª Tentativa: Título + Ano
    ) AS imdb_rating_concatenada,

    i_director.run_time_minutes,
    i_director.number_votes

    
FROM netflix_directors_exploded AS n
-- JOIN 1: Título + Diretor (Alta Precisão)
LEFT JOIN imdb_full_directors AS i_director
    ON n.title_clean = i_director.imdb_title_clean
    AND n.director_clean = i_director.imdb_director_clean

-- JOIN 2: Título + Ano (Média Precisão)
LEFT JOIN imdb_full_year AS i_year
    ON n.title_clean = i_year.imdb_title_clean
    AND jaro_winkler_similarity(n.title_clean, i_year.imdb_title_clean) > 0.8

GROUP BY
    n.show_id, n.type, n.title, n.director, n.cast, n.country,
    n.date_added, n.release_year, n.rating, n.duration,
    n.listed_in, n.description, n.streaming, n.genres_processed, 
    i_director.run_time_minutes, i_director.number_votes


HAVING
    COALESCE(AVG(i_director.imdb_rating), AVG(i_year.imdb_rating)) IS NOT NULL
"""

print("Executando a consulta de 'merge' (isso pode levar alguns minutos)...")
# Executa a consulta e salva o resultado em um novo CSV
con.execute(f"""
    COPY ({merge_query})
    TO '{output_csv_file}'
    WITH (HEADER 1, DELIMITER ',')
""")
print(f"\nSucesso! Arquivo salvo como: '{output_csv_file}'")

# --- Verificação ---
print("Verificando quantos títulos conseguimos concatenar...")

result = con.execute(f"""
    SELECT
        COUNT(*) AS total_titulos,
        COUNT(imdb_rating_concatenada) AS titulos_com_nota
    FROM read_csv_auto('{output_csv_file}', header=True)
""").df()

total = df_streaming.shape[0]
matched = result['titulos_com_nota'].iloc[0]
percent = (matched / total) * 100 if total > 0 else 0

print("\n--- Relatório de Concatenação ---")
print(f"Total de títulos no seu CSV: {total}")
print(f"Títulos que receberam nota do IMDb: {matched}")
print(f"Taxa de sucesso da concatenação: {percent:.2f}%")


#df = pd.read_csv(output_csv_file)
#df.info()

In [ ]:
# 1. Carregar os dados
imdb_data_dir = '../data/'
output_csv_file = os.path.join(imdb_data_dir, 'filmes_series_imdb.csv')
df = pd.read_csv(output_csv_file)

# Função auxiliar para limpeza (DRY - Don't Repeat Yourself)
def limpar_coluna_numerica(series):
    # Converte para string, remove '\n' e espaços extras
    s = series.astype(str).str.replace(r'\n', '', regex=True).str.strip()
    # Converte para número, transformando erros (como 'nan' string) em NaN real
    return pd.to_numeric(s, errors='coerce')

# 2. Aplica a limpeza
print("Limpando colunas...")
df['run_time_minutes'] = limpar_coluna_numerica(df['run_time_minutes'])
df['number_votes'] = limpar_coluna_numerica(df['number_votes'])

# 3. Tratamento de NaN (Valores Nulos)

# Usamos mediana porque a média é sensível a outliers (filmes de 10h ou 1min)
mediana_tempo = df['run_time_minutes'].median()
df['run_time_minutes'] = df['run_time_minutes'].fillna(mediana_tempo)
print(f"Valores nulos em 'run_time_minutes' preenchidos com a mediana: {mediana_tempo}")

# Se não tem votos registrados, assumir 0 é uma escolha segura e lógica.
df['number_votes'] = df['number_votes'].fillna(0)
print("Valores nulos em 'number_votes' preenchidos com 0.")

# 4. Verificação Final
print("\nInfo após tratamento:")
print(df[['run_time_minutes', 'number_votes']].info())
print(df[['run_time_minutes', 'number_votes']].head())

# 5. Salvar
df.to_csv(output_csv_file, index=False)